# Демонстрация отбора признаков с помощью метода Target Permutation

## Структура ноутбука:
- Демонстрация методов и атрибутов библиотеки Target Permutation для задач классификации
- Применение библиотеки Target Permutation для задач классификации
    * Проверка Target Permutation на искусственных данных
    * Проверка Target Permutation на данных (Титаник)
    
    
- Применение библиотеки Target Permutation для задач регрессии
    * Проверка Target Permutation на искусственных данных
    * Проверка Target Permutation на данных (Стоимость жилья)    

##  Подгрузим необходимые бибиотеки

In [1]:
# расширяем поле, убираем предупреждения
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:95% !important;}</style>'))

import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install PDPbox==0.2.0

In [3]:
# Стандартные библиотеки для анализа
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, mean_absolute_percentage_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import matplotlib.pylab as pl

In [4]:
import sys
sys.path.append('../')

# Наша разработанная библиотека
from autobinary import SentColumns, CatBoostEncoder, TargetPermutationSelection, base_pipe #, utils_target_permutation
# подгрузим вспомогательные функции
from autobinary import utils_target_permutation
# from utils_target_permutation import select_estimator, geom_reg_list, aggregated_feature_selection, intersection_set

In [5]:
import sklearn as sk
import xgboost as xgb
import matplotlib as mplot
import autobinary as ab

print('sklearn: ', sk.__version__)
print('xgboost: ', xgb.__version__)
print('matplotlib: ', mplot.__version__)
print('autobinary: ', ab.__version__)

sklearn:  0.24.2
xgboost:  1.5.2
matplotlib:  3.1.0
autobinary:  1.0.5


## Генерация искусственных данных

In [6]:
from sklearn.datasets import make_classification

X,y = make_classification(
    n_samples=10000,
    n_features=10,
    n_informative=5,
    n_classes=2,
    n_clusters_per_class=2,
    class_sep=2,
    weights=[0.5, 0.5],
    random_state=2022)

columns = ['A','B','C','D','E','F','G','H','I','J']
sample = pd.DataFrame(X, columns=columns)
sample['target'] = y
target = 'target'

X_train, X_valid, y_train, y_valid = train_test_split(
    sample.drop(columns=target), 
    sample[target], 
    test_size=0.3, 
    random_state=2022)

In [7]:
num_columns = columns
cat_columns = []
target = ['target']

#pipeline
prep_pipe = base_pipe(
    num_columns=num_columns,
    cat_columns=cat_columns)

## Демострация методов и атрибутов Target Permutation

Описание: Класс **TargetPermutationSelection** предназначен для отбора признаков с помощью перемешивания таргета и оценки предсказательной способности фичей.

Последовательность действий алгоритма:
- `1)` Происходит обучение модели с оригинальным тарегетом (benchmark-модель);
- `2)` Происходит расчет метрики benchmark-модели;
- `3)` Происходит перемешивание таргета;
- `4)` Происходит обучение модели с перемешаным таргетом (target permutation-модель);
- `5)` Происходит расчет метрики target permutation-модели;
- `6)` Расчет разности метрик 2) и 5)
- `7)` Происходит отбор признаков

In [8]:
from xgboost import XGBClassifier

params_xgb = {
    'eta':0.01,
    'n_estimators':800,
    'subsample':0.9,
    'colsample_bytree':0.6,
    'max_depth':6,
    'objective':'binary:logistic',
    'use_label_encoder':False,
    'n_jobs':-1,
    'random_state':42}

fit_params_xgb = {
    'early_stopping_rounds':100, 
    'eval_metric':['logloss', 'aucpr', 'auc'],
    'verbose':0}

estimator = XGBClassifier(**params_xgb)

In [9]:
# инициализация метода
model = TargetPermutationSelection(
    main_estimator=estimator, 
    main_fit_params=fit_params_xgb,
    main_prep_pipe=prep_pipe,
    main_features=num_columns + cat_columns,

    X_train=sample[num_columns + cat_columns],
    y_train=sample[target], 
    main_metric='gini',
    model_type='xgboost')

# метод для вычисления важностей фичей
model.calculate_permutation(test_size=0.3)

============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  61
gini на обучающей выборке: 99.449
gini на проверочной выборке: 98.993
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  102
gini на обучающей выборке: 67.918
gini на проверочной выборке: 1.051


,index,importance_benchmark,importance_permut,final_importance
0,F,231.403839,4.202850,227.200989
1,A,64.806770,4.327067,60.479703
2,J,62.913349,4.319738,58.593611
3,C,32.646683,4.031488,28.615195
4,G,29.843172,4.974428,24.868744
5,B,19.961395,4.079677,15.881718
6,D,8.966094,4.603549,4.362545
7,E,1.769365,4.535268,-2.765903
8,H,1.713513,4.520055,-2.806542
10,random_feature,0.810481,4.466777,-3.656296


In [10]:
# отбор фичей с помощью пересечения множеств
model.select_features(select_type='random_feature', cut_off=0.0) # для отбора фичей, которые результативнее рандомной фичи
# model.select_features(select_type='any', cut_off=0.0) # для отбора фичей, важность которых больше cut_off

10 признаков было до Target Permutation
9 признаков после Target Permutation


['F', 'A', 'J', 'C', 'G', 'B', 'D', 'E', 'H']

In [40]:
# атрибуты
model.selected_features # отобранные фичи

['TotalBsmtSF',
 'FullBath',
 'BldgType',
 'CentralAir',
 'PoolArea',
 'GarageQual',
 'Fence',
 'LotFrontage',
 'SaleType']

In [41]:
model.df_benchmark_fi # датафрейм с важностями для benchmark-модели

,index,importance_benchmark
0,TotalBsmtSF,0.565400
1,FullBath,0.201392
2,LotArea,0.076055
3,BldgType,0.046845
4,LotFrontage,0.032027
5,random_feature,0.026229
6,SaleType,0.020573
7,GarageQual,0.017909
8,CentralAir,0.011023
9,Fence,0.001798


In [42]:
model.df_permutation_fi # датафрейм с важностями для target permutation-модели

,index,importance_permut
0,LotArea,0.249514
1,TotalBsmtSF,0.226762
2,random_feature,0.184434
3,SaleType,0.160461
4,LotFrontage,0.093191
5,Fence,0.028229
6,GarageQual,0.020169
7,BldgType,0.016104
8,FullBath,0.011552
9,CentralAir,0.007867


In [43]:
model.df_fi_final # финальный датафрейм важностей фичей  

,index,importance_benchmark,importance_permut,final_importance
0,TotalBsmtSF,0.565400,0.226762,0.338638
1,FullBath,0.201392,0.011552,0.189840
3,BldgType,0.046845,0.016104,0.030741
8,CentralAir,0.011023,0.007867,0.003157
10,PoolArea,0.000748,0.001717,-0.000969
7,GarageQual,0.017909,0.020169,-0.002260
9,Fence,0.001798,0.028229,-0.026431
4,LotFrontage,0.032027,0.093191,-0.061164
6,SaleType,0.020573,0.160461,-0.139888
5,random_feature,0.026229,0.184434,-0.158205


## Демонстрация метода на искусственных данных для задач классификации

In [12]:
# параметры для задачи классификации для разных эстиматоров
params_xgb = {'eta':0.01,
              'n_estimators':800,
              'subsample':0.9,
              'colsample_bytree':0.6,
              'max_depth':6,
              'objective':'binary:logistic',
              'use_label_encoder':False,
              'n_jobs':-1,
              'random_state':42}

params_ctb = {'depth':10,
              'subsample':0.33,
              'learning_rate':0.01,
              'loss_function':'Logloss',
              'random_state':42,
              'verbose':0}

params_lgb = {'learning_rate':0.01,
              'subsample':0.9,
              'colsample_bytree':0.6,
              'objective':'binary',
              'metric':'binary_logloss',
              'n_jobs':-1,
              'random_state':42,
              'verbose':0,
              'force_col_wise':True}

params_dt = {'criterion':'gini',
             'max_depth':5,
             'random_state':42}

params_rf = {'criterion':'gini',
             'max_depth':5,
             'random_state':42,
             'n_estimators':50}

fit_params_xgb = {
    'early_stopping_rounds':100, 
    'eval_metric':['logloss', 'aucpr', 'auc'],
    'verbose':0}

fit_params_ctb = {
    'use_best_model':True,
    'early_stopping_rounds':200, 
    'verbose':0,
    'plot':False}

fit_params_lgb = {
    'early_stopping_rounds':100, 
    'eval_metric':['logloss', 'auc'],
    'verbose':0}

fit_params_rf = {}

fit_params_dt = {}


params_list = [params_xgb, 
               params_ctb, 
               params_lgb, 
               params_dt, 
               params_rf
              ]


fit_params_list = [fit_params_xgb, 
                   fit_params_ctb, 
                   fit_params_lgb, 
                   fit_params_dt, 
                   fit_params_rf
                  ]

models_list = ['xgboost', 
               'catboost', 
               'lightboost', 
               'decisiontree', 
               'randomforest'
              ] 

In [13]:
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator('classification', models_list[i], params_list[i])
    
    print('\n                                  Модель: ', models_list[i])
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='gini',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('\n     Модель: ', models_list[i])
    features_dict_target_perm[models_list[i]] = model.select_features(select_type='tyta')
    df_importance[models_list[i]] = model.df_fi_final


                                  Модель:  xgboost
============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  61
gini на обучающей выборке: 99.435
gini на проверочной выборке: 98.995
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  28
gini на обучающей выборке: 55.738
gini на проверочной выборке: -1.648

     Модель:  xgboost
10 признаков было до Target Permutation
7 признаков после Target Permutation

                                  Модель:  catboost
============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  937
gini на обучающей выборке: 99.994
gini на проверочной выборке: 98.999
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  68
gini на обучающей выборке: 85.535
gini на проверочной выборке: 4.396

     Модель:  catboost
10 признаков было до Target Permutati

In [14]:
features_dict_target_perm

{'xgboost': ['F', 'A', 'J', 'C', 'G', 'B', 'D'],
 'catboost': ['F', 'G', 'A', 'C', 'B', 'J'],
 'lightboost': ['F', 'J', 'A', 'C', 'G', 'B', 'D'],
 'decisiontree': ['F', 'G'],
 'randomforest': ['F', 'A', 'J']}

In [15]:
print(utils_target_permutation.intersection_set(features_dict_target_perm, 3))

{'C', 'J', 'B', 'G', 'F', 'A'}


In [16]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 4 фичей


,features,score
6,F,100.0000
4,A,68.4000
3,G,62.2440
1,J,62.2098


In [17]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 7 фичей


,features,score
6,F,100.00000
4,A,68.40000
3,G,62.24400
1,J,62.20980
0,C,43.74000
2,B,36.74160
5,D,21.25764


## Демонстрация метода на данных Титаник

In [18]:
sample = pd.read_csv('../data/train_titanic.csv')

print('Размерность данных:', sample.shape)
sample.head(2)

Размерность данных: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [19]:
num_columns = [
    'Age', 'Fare'
]

cat_columns = [
    'Pclass', 'Sex', 'Embarked', 'SibSp', 'Parch'
]

target = ['Survived']

prep_pipe = base_pipe(
    num_columns=num_columns, 
    cat_columns=cat_columns)

In [20]:
%%time
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator('classification', models_list[i], params_list[i])
    
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='gini',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('Модель: ', models_list[i], '\n')
    features_dict_target_perm[models_list[i]] = model.select_features()
    df_importance[models_list[i]] = model.df_fi_final

============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  0
gini на обучающей выборке: 68.352
gini на проверочной выборке: 72.500
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  139
gini на обучающей выборке: 84.057
gini на проверочной выборке: 12.101
Модель:  xgboost 

7 признаков было до Target Permutation
7 признаков после Target Permutation
============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  299
gini на обучающей выборке: 95.878
gini на проверочной выборке: 70.976
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  48
gini на обучающей выборке: 85.600
gini на проверочной выборке: 2.078
Модель:  catboost 

7 признаков было до Target Permutation
6 признаков после Target Permutation
============================== Обучение benchmark-модели! =================

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

gini на обучающей выборке: 85.951
gini на проверочной выборке: 70.976
============================== Обучение target permutation-модели! ==============================
gini на обучающей выборке: 62.972
gini на проверочной выборке: 2.660
Модель:  randomforest 

7 признаков было до Target Permutation
7 признаков после Target Permutation
CPU times: user 32.1 s, sys: 1.53 s, total: 33.7 s
Wall time: 4.3 s


In [21]:
features_dict_target_perm

{'xgboost': ['Sex', 'Pclass', 'Parch', 'SibSp', 'Fare', 'Embarked', 'Age'],
 'catboost': ['Sex', 'Pclass', 'Fare', 'Parch', 'SibSp', 'Age'],
 'lightboost': ['Sex', 'Pclass', 'Fare', 'Parch', 'Age', 'Embarked', 'SibSp'],
 'decisiontree': ['Sex',
  'Pclass',
  'Parch',
  'Embarked',
  'Fare',
  'Age',
  'SibSp'],
 'randomforest': ['Sex',
  'Pclass',
  'Fare',
  'Parch',
  'Embarked',
  'Age',
  'SibSp']}

In [22]:
print(utils_target_permutation.intersection_set(features_dict_target_perm, 3))

{'Embarked', 'SibSp', 'Sex', 'Fare', 'Age', 'Pclass', 'Parch'}


In [23]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 4 фичей


,features,score
3,Sex,100.000
5,Pclass,90.000
6,Parch,76.140
4,Fare,74.844


In [24]:
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 7 фичей


,features,score
3,Sex,100.00000
5,Pclass,90.00000
6,Parch,76.14000
4,Fare,74.84400
0,SibSp,59.58846
1,Age,59.18022
2,Embarked,51.32160


# Задача Регрессии 

## Генерация искуственных данных

In [25]:
from sklearn.datasets import make_regression

X,y = make_regression(n_samples=10000,
                      n_features=10,
                      n_informative=5,
                      n_targets=1,
                      noise=25,
                      random_state=2022)

columns = ['A','B','C','D','E','F','G','H','I','J']
sample = pd.DataFrame(X, columns=columns)
sample['target'] = y
target = 'target'

In [26]:
num_columns = columns
cat_columns = []
target = ['target']

#pipline
prep_pipe = base_pipe(num_columns=num_columns,
                     cat_columns=cat_columns)

In [27]:
# Определение стандартных гиперпараметров моделей и список моделей
params_xgb = {
    'eta':0.01,
    'n_estimators':1000,
    'subsample':0.9,
    'colsample_bytree':0.6,
    'max_depth':6,
    'objective':'reg:squarederror',
    'n_jobs':-1,
    'random_state':42}

params_ctb = {
    'learning_rate':0.01,
    'iterations':1000,
    'subsample':0.9,
    'colsample_bylevel':0.9,
    'depth':6,
    'loss_function':'MAPE',
    'thread_count':-1,
    'random_state':42,
    'custom_metric':'MAE',
    'verbose':0}

params_lgb = {
    'learning_rate':0.01,
    'n_estimators':1000,
    'subsample':0.9,
    'colsample_bytree':0.6,
    'max_depth':6,
    'objective':'regression',
    'n_jobs':-1,
    'random_state':42}

params_dt = {'max_depth':5,
             'random_state':42}

params_rf = {'max_depth':5,
             'random_state':42,
             'n_estimators':50}

fit_params_xgb = {
    'early_stopping_rounds':200, 
    'eval_metric':'rmse',
    'verbose':0}

fit_params_ctb = {
    'use_best_model':True,
    'early_stopping_rounds':200, 
    'verbose':0,
    'plot':False}

fit_params_lgb = {
    'early_stopping_rounds':200, 
    'eval_metric':'rmse',
    'verbose':0}

fit_params_rf = {}

fit_params_dt = {}


params_list = [params_xgb, 
               params_ctb, 
               params_lgb, 
               params_dt, 
               params_rf
              ]

fit_params_list = [fit_params_xgb, 
                   fit_params_ctb, 
                   fit_params_lgb, 
                   fit_params_dt, 
                   fit_params_rf]

models_list = ['xgboost', 
               'catboost', 
               'lightboost', 
               'decisiontree', 
               'randomforest']

In [28]:
%%time
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator('regression', models_list[i], params_list[i])
    
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='mape',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('Модель: ', models_list[i], '\n')
    features_dict_target_perm[models_list[i]] = model.select_features()
    df_importance[models_list[i]] = model.df_fi_final

============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  999
mape на обучающей выборке: 0.811
mape на проверочной выборке: 1.126
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  14
mape на обучающей выборке: 1.014
mape на проверочной выборке: 1.065
Модель:  xgboost 

10 признаков было до Target Permutation
10 признаков после Target Permutation
============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  999
mape на обучающей выборке: 0.766
mape на проверочной выборке: 0.922
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  16
mape на обучающей выборке: 0.998
mape на проверочной выборке: 0.999
Модель:  catboost 

10 признаков было до Target Permutation
4 признаков после Target Permutation
============================== Обучение benchmark-модели! ====================

In [29]:
features_dict_target_perm

{'xgboost': ['G', 'E', 'J', 'A', 'B', 'C', 'D', 'H', 'F', 'I'],
 'catboost': ['G', 'E', 'J', 'A'],
 'lightboost': ['G', 'E', 'J', 'A', 'H'],
 'decisiontree': ['G', 'F', 'H', 'J'],
 'randomforest': ['G', 'E', 'J']}

In [30]:
print(utils_target_permutation.intersection_set(features_dict_target_perm , 3))

{'J', 'G', 'E', 'A', 'H'}


In [31]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 4 фичей


,features,score
3,G,100.00
1,J,79.38
4,E,72.00
5,A,43.74


In [32]:
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 10 фичей


,features,score
3,G,100.000000
1,J,79.380000
4,E,72.000000
5,A,43.740000
7,H,38.887938
9,F,26.609344
2,B,13.122000
0,C,11.809800
6,D,10.628820
8,I,7.748410


## Демонстрация метода на данных Стоимость жилья

In [33]:
sample = pd.read_csv('../data/train_houseprice.csv')

print('Размерность данных:', sample.shape)
sample.head(2)

Размерность данных: (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [34]:
num_columns = [
    'LotArea', 'LotFrontage', 'TotalBsmtSF', 'PoolArea', 'FullBath'
]

cat_columns = [
    'BldgType', 'CentralAir', 'GarageQual', 'Fence', 'SaleType'
]

target = ['SalePrice']

prep_pipe = base_pipe(
    num_columns=num_columns, 
    cat_columns=cat_columns)

In [35]:
%%time
features_dict_target_perm = dict()
df_importance = dict()

for i in range(len(params_list)):
    estimator = utils_target_permutation.select_estimator('regression', models_list[i], params_list[i])
    
    model = TargetPermutationSelection(
        main_estimator=estimator, 
        main_fit_params=fit_params_list[i],
        main_prep_pipe=prep_pipe,
        main_features=num_columns + cat_columns,
        
        X_train=sample[num_columns + cat_columns],
        y_train=sample[target], 
        main_metric='mape',
        model_type=models_list[i])
    
    model.calculate_permutation()
    
    print('Модель: ', models_list[i], '\n')
    features_dict_target_perm[models_list[i]] = model.select_features()
    df_importance[models_list[i]] = model.df_fi_final

============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  958
mape на обучающей выборке: 0.084
mape на проверочной выборке: 0.160
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  327
mape на обучающей выборке: 0.255
mape на проверочной выборке: 0.361
Модель:  xgboost 

10 признаков было до Target Permutation
10 признаков после Target Permutation
============================== Обучение benchmark-модели! ==============================
BEST ITERATION:  998
mape на обучающей выборке: 0.225
mape на проверочной выборке: 0.238
============================== Обучение target permutation-модели! ==============================
BEST ITERATION:  50
mape на обучающей выборке: 0.286
mape на проверочной выборке: 0.309
Модель:  catboost 

10 признаков было до Target Permutation
5 признаков после Target Permutation
============================== Обучение benchmark-модели! ===================

In [36]:
features_dict_target_perm

{'xgboost': ['FullBath',
  'TotalBsmtSF',
  'SaleType',
  'BldgType',
  'CentralAir',
  'GarageQual',
  'PoolArea',
  'LotArea',
  'Fence',
  'LotFrontage'],
 'catboost': ['TotalBsmtSF', 'FullBath', 'LotArea', 'GarageQual', 'PoolArea'],
 'lightboost': ['TotalBsmtSF',
  'FullBath',
  'LotArea',
  'LotFrontage',
  'SaleType',
  'BldgType'],
 'decisiontree': ['TotalBsmtSF',
  'FullBath',
  'BldgType',
  'GarageQual',
  'CentralAir',
  'PoolArea',
  'Fence',
  'SaleType'],
 'randomforest': ['TotalBsmtSF',
  'FullBath',
  'BldgType',
  'CentralAir',
  'PoolArea',
  'GarageQual',
  'Fence',
  'LotFrontage',
  'SaleType']}

In [37]:
print(utils_target_permutation.intersection_set(features_dict_target_perm))

{'LotFrontage', 'PoolArea', 'SaleType', 'LotArea', 'BldgType', 'CentralAir', 'FullBath', 'Fence', 'GarageQual', 'TotalBsmtSF'}


In [38]:
# отбор фичей при помощи взвешенной частоты фичей
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = True)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобраны фичи, которые имеют скор больше среднего.
Отобрано 3 фичей


,features,score
9,TotalBsmtSF,98.0000
6,FullBath,92.0000
4,BldgType,58.7898


In [39]:
utils_target_permutation.aggregated_feature_selection(feat_dist = features_dict_target_perm, mean_score_select = False)

Веса для первый фичи для каждой модели выбраны одинаковыми => все модели имеют одинаковый вес.
Отобрано 10 фичей


,features,score
9,TotalBsmtSF,98.000000
6,FullBath,92.000000
4,BldgType,58.789800
8,GarageQual,52.779600
1,PoolArea,48.682620
2,SaleType,47.497282
3,LotArea,41.965938
5,CentralAir,40.824000
0,LotFrontage,31.894348
7,Fence,29.866984
